# 🗺️ Histoire-Géographie avec GenAI

**Module :** Examples Sectoriels - Histoire-Géographie  
**Niveau :** 🟡 Intermédiaire  
**Technologies :** DALL-E 3, GPT-5 Vision  
**Durée estimée :** 30 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Générer des cartes historiques avec contexte
- [ ] Créer des reconstructions visuelles de sites historiques
- [ ] Produire des visualisations géographiques thématiques
- [ ] Concevoir des timelines historiques illustrées
- [ ] Adapter le contenu visuel au niveau éducatif

## 📚 Prérequis

- Notebooks Foundation (01-1, 01-2) complétés
- Connaissances historiques et géographiques de base

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

notebook_mode = "interactive"
skip_widgets = False
debug_level = "INFO"

# Configuration contenu
history_period = "all"  # "ancient", "medieval", "modern", "contemporary", "all"
geography_focus = "world"  # "world", "europe", "france", "local"
image_style = "educational"  # "educational", "artistic", "documentary"
image_quality = "hd"

# Sujets
history_topics = ["roman_empire", "french_revolution", "world_war_1"]
geography_topics = ["world_map", "climate_zones", "mountain_ranges"]

save_resources = True
export_lesson_pack = True

In [ ]:
# Setup environnement
import os
import sys
import json
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import logging

GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'history_geography'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('history_geography')

openrouter_key = os.getenv('OPENROUTER_API_KEY')
if not openrouter_key:
    raise ValueError("❌ OPENROUTER_API_KEY manquante")

api_base_url = "https://openrouter.ai/api/v1"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "HTTP-Referer": "https://coursia.myia.io",
    "X-Title": "CoursIA History Geography",
    "Content-Type": "application/json"
}

print(f"🗺️  Histoire-Géographie avec GenAI")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Période: {history_period}, Focus: {geography_focus}")
print(f"✅ Environnement configuré")

## 🏛️ Section 1: Cartes Historiques

Génération de cartes avec contexte historique et évolutions territoriales.

In [ ]:
# Templates cartes historiques
historical_maps = {
    "roman_empire": {
        "title": "🏛️ Empire Romain (117 ap. J-C)",
        "prompt": """Create an educational historical map of the Roman Empire at its peak (117 AD):
- Europe, North Africa, Middle East visible
- Roman territory clearly marked in distinct color (red/pink)
- Major provinces labeled (Britannia, Gallia, Hispania, Italia, etc.)
- Important cities: Rome, Constantinople, Alexandria, Carthage
- Major roads (Via Appia, etc.) as thin lines
- Borders with neighboring peoples (Germania, Parthia, etc.)
- Legend showing provinces, cities, roads
- Compass rose
- Scale bar

Style: Clean educational map. Roman territory in red, seas in blue, land in beige.
Clear labels in readable font. Professional history textbook quality.""",
        "period": "Antiquité",
        "grade_level": "6ème"
    },
    "french_revolution_map": {
        "title": "⚔️ France Révolutionnaire (1789-1799)",
        "prompt": """Create an educational map of France during the Revolutionary period (1789-1799):
- France's borders clearly defined
- Major revolutionary departments marked
- Important cities of revolutionary events (Paris, Lyon, Marseille, Bordeaux)
- Areas of counter-revolutionary activity (Vendée) in different color
- Routes of revolutionary armies
- Neighboring countries and their stance
- Legend explaining symbols and colors

Style: Historical educational map. Use revolutionary colors (blue, white, red accents).
Clear labels. White background. Professional history textbook style.""",
        "period": "Époque Moderne",
        "grade_level": "4ème"
    },
    "world_exploration": {
        "title": "🌍 Grandes Découvertes (XV-XVIe siècles)",
        "prompt": """Create an educational map showing major exploration routes (15th-16th centuries):
- World map with continents
- Major exploration routes marked with different colored lines:
  * Columbus (1492) - red line
  * Vasco da Gama (1498) - blue line
  * Magellan (1519-1522) - green line
- Starting points (Spain, Portugal) and destinations
- Trade routes and colonies established
- Compass rose
- Legend with explorer names and dates

Style: Vintage map aesthetic. Aged parchment background. Clear colored routes.
Professional historical atlas quality.""",
        "period": "Renaissance",
        "grade_level": "5ème"
    }
}

print("\n🏛️ CARTES HISTORIQUES")
print("=" * 50)
for key, map_data in historical_maps.items():
    print(f"\n{map_data['title']}")
    print(f"   📅 Période: {map_data['period']}")
    print(f"   🎓 Niveau: {map_data['grade_level']}")

In [ ]:
# Fonction génération cartes
def generate_historical_visual(prompt: str,
                              title: str,
                              category: str,
                              quality: str = "hd") -> Dict[str, Any]:
    """Génère une visualisation historique/géographique."""
    
    payload = {
        "model": "openai/dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1792x1024",
        "quality": quality,
        "style": "natural",
        "response_format": "url"
    }
    
    try:
        print(f"\n🎨 Génération: {title}")
        response = requests.post(
            f"{api_base_url}/images/generations",
            headers=headers,
            json=payload,
            timeout=90
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ Généré en {response.elapsed.total_seconds():.1f}s")
            
            return {
                "success": True,
                "image_url": result["data"][0]["url"],
                "title": title,
                "category": category,
                "revised_prompt": result["data"][0].get("revised_prompt", prompt),
                "timestamp": datetime.now().isoformat()
            }
        else:
            return {"success": False, "error": response.json().get("error", {}).get("message")}
    except Exception as e:
        return {"success": False, "error": str(e)}

def save_and_display(result: Dict[str, Any]) -> Optional[Image.Image]:
    """Télécharge, affiche et sauvegarde."""
    if not result.get('success'):
        print(f"   ❌ Échec: {result.get('error')}")
        return None
    
    try:
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            plt.figure(figsize=(16, 9))
            plt.imshow(image)
            plt.title(result['title'], fontsize=14, fontweight='bold', pad=20)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            if save_resources:
                cat_dir = OUTPUT_DIR / result['category']
                cat_dir.mkdir(exist_ok=True)
                
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                safe_title = result['title'].replace(' ', '_')[:50]
                filename = f"{safe_title}_{timestamp}.png"
                
                image.save(cat_dir / filename)
                print(f"   💾 Sauvegardé: {filename}")
            
            return image
    except Exception as e:
        print(f"   ❌ Erreur: {str(e)[:100]}")
    
    return None

# Génération: Empire Romain
if history_period in ["ancient", "all"]:
    print("\n" + "=" * 60)
    print("🏛️ GÉNÉRATION: EMPIRE ROMAIN")
    print("=" * 60)
    
    roman_map = historical_maps["roman_empire"]
    roman_result = generate_historical_visual(
        prompt=roman_map["prompt"],
        title=roman_map["title"],
        category="history_maps",
        quality=image_quality
    )
    
    if roman_result['success']:
        roman_image = save_and_display(roman_result)
        print(f"\n💡 Usages pédagogiques:")
        print(f"   • Cours d'Histoire {roman_map['grade_level']}")
        print(f"   • Étude expansion romaine")
        print(f"   • Exercices de localisation")
        print(f"   • Comparaison avec l'Europe actuelle")

## 🏰 Section 2: Reconstructions Historiques

Visualisation de sites et monuments historiques tels qu'ils étaient.

In [ ]:
# Templates reconstructions
historical_reconstructions = {
    "colosseum": {
        "title": "🏛️ Colisée de Rome (Ier siècle)",
        "prompt": """Create a historically accurate reconstruction of the Colosseum in ancient Rome (1st century AD):
- Exterior view showing the complete amphitheater
- Original white marble and travertine stone
- All levels intact (ground, first, second, third floors)
- Awning system (velarium) visible
- Crowds of people in Roman togas
- Surrounding Roman architecture visible
- Clear blue sky
- Vibrant, lived-in appearance

Style: Historical reconstruction, photorealistic. Show the building in its prime.
Bright daylight. Educational history documentary quality.""",
        "period": "Antiquité",
        "educational_value": "Comprendre l'architecture romaine et les loisirs"
    },
    "versailles_construction": {
        "title": "👑 Château de Versailles (XVIIe siècle)",
        "prompt": """Create a historical reconstruction of the Palace of Versailles during Louis XIV's reign (17th century):
- Main palace facade with its distinctive architecture
- Elaborate gardens with fountains active
- Courtiers in period clothing (elaborate dresses, wigs)
- Carriages arriving at the palace
- Golden details and decorations prominent
- Formal French garden layout visible
- Sunny day showing the palace in its glory

Style: Historical accuracy. Baroque opulence. Rich colors and gold accents.
Educational documentary quality showing royal life.""",
        "period": "Ancien Régime",
        "educational_value": "Visualiser le mode de vie de la monarchie absolue"
    },
    "medieval_castle": {
        "title": "🏰 Château-Fort Médiéval (XIIIe siècle)",
        "prompt": """Create a historically accurate medieval castle (13th century):
- Stone castle with multiple towers and battlements
- Moat with water surrounding the castle
- Drawbridge in raised position
- Banners flying from towers
- Medieval village visible nearby
- People in medieval clothing around the castle
- Guards on the walls
- Countryside landscape

Style: Medieval period accuracy. Stone textures realistic.
Daylight showing defensive architecture clearly. Educational medieval history quality.""",
        "period": "Moyen-Âge",
        "educational_value": "Comprendre l'architecture défensive médiévale"
    }
}

print("\n🏰 RECONSTRUCTIONS HISTORIQUES")
print("=" * 50)
for key, recon in historical_reconstructions.items():
    print(f"\n{recon['title']}")
    print(f"   📅 Période: {recon['period']}")
    print(f"   🎯 Valeur: {recon['educational_value']}")

# Génération: Colisée
if history_period in ["ancient", "all"]:
    print("\n" + "=" * 60)
    print("🏛️ GÉNÉRATION: COLISÉE DE ROME")
    print("=" * 60)
    
    colosseum = historical_reconstructions["colosseum"]
    colosseum_result = generate_historical_visual(
        prompt=colosseum["prompt"],
        title=colosseum["title"],
        category="historical_reconstructions",
        quality=image_quality
    )
    
    if colosseum_result['success']:
        colosseum_image = save_and_display(colosseum_result)
        print(f"\n💡 Applications:")
        print(f"   • Cours sur la civilisation romaine")
        print(f"   • Comparaison passé/présent")
        print(f"   • Étude architecture antique")

## 🌍 Section 3: Géographie Thématique

Cartes et visualisations géographiques (relief, climat, ressources).

In [ ]:
# Templates géographie
geography_visuals = {
    "world_climate": {
        "title": "🌡️ Zones Climatiques Mondiales",
        "prompt": """Create an educational world map showing climate zones:
- World map with all continents clearly visible
- Climate zones color-coded:
  * Tropical (red/orange)
  * Dry/Desert (yellow/brown)
  * Temperate (green)
  * Continental (light blue)
  * Polar (white/ice blue)
- Clear legend explaining each climate zone
- Major ocean currents (arrows)
- Equator, Tropics, and Polar circles marked
- Labels for major geographic features

Style: Educational geography textbook. Clear colors. White background.
Professional cartographic quality.""",
        "theme": "Climat",
        "grade_level": "5ème/2nde"
    },
    "mountain_ranges": {
        "title": "⛰️ Grandes Chaînes de Montagnes",
        "prompt": """Create an educational 3D relief map showing major mountain ranges:
- World view highlighting mountain systems:
  * Himalayas (highest, snow-capped)
  * Andes (long chain)
  * Rocky Mountains
  * Alps
  * Atlas Mountains
- Relief shading showing elevation
- Color gradient from low (green) to high (brown to white)
- Labels for major peaks (Everest, K2, Mont Blanc, etc.)
- Scale showing elevation
- Oceans in blue

Style: 3D relief map. Physical geography style. Clear elevation coloring.
Educational atlas quality.""",
        "theme": "Relief",
        "grade_level": "6ème/5ème"
    },
    "france_regions": {
        "title": "🇫🇷 Régions de France",
        "prompt": """Create an educational map of France showing all regions:
- France with clear borders
- All 13 metropolitan regions distinctly colored
- Regional capitals marked with stars
- Major cities labeled (Paris, Lyon, Marseille, etc.)
- Neighboring countries labeled
- Seas and ocean labeled (Manche, Atlantique, Méditerranée)
- Rivers (Loire, Seine, Rhône, Garonne)
- Legend with region names
- Compass rose and scale

Style: Clean administrative map. Pastel colors for regions.
White background. Professional French geography textbook quality.""",
        "theme": "Administration",
        "grade_level": "CM1/CM2"
    }
}

print("\n🌍 GÉOGRAPHIE THÉMATIQUE")
print("=" * 50)
for key, geo in geography_visuals.items():
    print(f"\n{geo['title']}")
    print(f"   🗺️  Thème: {geo['theme']}")
    print(f"   🎓 Niveau: {geo['grade_level']}")

# Génération: Zones Climatiques
if geography_focus in ["world", "all"]:
    print("\n" + "=" * 60)
    print("🌡️ GÉNÉRATION: ZONES CLIMATIQUES")
    print("=" * 60)
    
    climate = geography_visuals["world_climate"]
    climate_result = generate_historical_visual(
        prompt=climate["prompt"],
        title=climate["title"],
        category="geography",
        quality=image_quality
    )
    
    if climate_result['success']:
        climate_image = save_and_display(climate_result)
        print(f"\n💡 Usages:")
        print(f"   • Cours de géographie {climate['grade_level']}")
        print(f"   • Étude climats et biomes")
        print(f"   • Lien avec écosystèmes")

## 📅 Section 4: Timelines Historiques Illustrées

Création de frises chronologiques visuelles avec événements clés.

In [ ]:
# Template timeline
timeline_prompt = """Create an educational illustrated timeline of the French Revolution (1789-1799):

Structure:
- Horizontal timeline from left (1789) to right (1799)
- Major events marked with dates and small illustrations:
  * 1789: Storming of Bastille (fortress image)
  * 1791: Constitutional Monarchy (document)
  * 1792: First Republic (tricolor flag)
  * 1793: Reign of Terror (guillotine silhouette)
  * 1794: Fall of Robespierre
  * 1799: Napoleon's coup (Napoleon figure)

- Timeline bar in blue-white-red (French colors)
- Each event has:
  * Date clearly marked
  * Small iconic illustration
  * Brief label
- Clean, organized layout
- Educational poster style

Style: Infographic quality. Clear, colorful, educational.
White background. Professional history education poster."""

print("\n📅 TIMELINE ILLUSTRÉE")
print("=" * 50)
print("\n🎨 Génération: Révolution Française (1789-1799)")

timeline_result = generate_historical_visual(
    prompt=timeline_prompt,
    title="📅 Timeline Révolution Française",
    category="timelines",
    quality=image_quality
)

if timeline_result['success']:
    timeline_image = save_and_display(timeline_result)
    print(f"\n✅ Timeline générée")
    print(f"\n💡 Applications pédagogiques:")
    print(f"   • Support visuel cours d'histoire")
    print(f"   • Affichage classe")
    print(f"   • Révisions chronologie")
    print(f"   • Exercices de datation")
    print(f"   • Compréhension enchainement événements")

## 📦 Section 5: Pack Pédagogique Histoire-Géo

Export complet pour enseignants.

In [ ]:
if export_lesson_pack:
    print("\n📦 EXPORT PACK PÉDAGOGIQUE")
    print("=" * 50)
    
    pack_dir = OUTPUT_DIR / f"history_geo_pack_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    pack_dir.mkdir(exist_ok=True)
    
    (pack_dir / 'maps').mkdir(exist_ok=True)
    (pack_dir / 'reconstructions').mkdir(exist_ok=True)
    (pack_dir / 'geography').mkdir(exist_ok=True)
    (pack_dir / 'timelines').mkdir(exist_ok=True)
    (pack_dir / 'lesson_plans').mkdir(exist_ok=True)
    
    print(f"\n📁 Structure créée: {pack_dir.name}")
    
    # Collecte ressources générées
    resources = []
    if 'roman_result' in locals() and roman_result.get('success'):
        resources.append(('roman', roman_result, historical_maps['roman_empire']))
    if 'colosseum_result' in locals() and colosseum_result.get('success'):
        resources.append(('colosseum', colosseum_result, historical_reconstructions['colosseum']))
    if 'climate_result' in locals() and climate_result.get('success'):
        resources.append(('climate', climate_result, geography_visuals['world_climate']))
    if 'timeline_result' in locals() and timeline_result.get('success'):
        resources.append(('timeline', timeline_result, {'period': 'Révolution', 'educational_value': 'Chronologie'}))
    
    print(f"\n📊 {len(resources)} ressources à exporter")
    
    # Export
    pack_index = {
        "title": "Pack Pédagogique Histoire-Géographie",
        "created": datetime.now().isoformat(),
        "period": history_period,
        "focus": geography_focus,
        "resources": []
    }
    
    for key, result, template in resources:
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            # Déterminer le sous-dossier
            subfolder = result['category']
            img_file = pack_dir / subfolder / f"{key}.png"
            image.save(img_file)
            
            pack_index['resources'].append({
                "id": key,
                "title": result['title'],
                "category": result['category'],
                "file": f"{subfolder}/{key}.png",
                "period": template.get('period', 'N/A'),
                "value": template.get('educational_value', template.get('theme', 'N/A'))
            })
            
            print(f"   ✅ {result['title']}")
    
    # README
    readme = f"""# Pack Pédagogique Histoire-Géographie

**Créé:** {datetime.now().strftime('%Y-%m-%d')}
**Période:** {history_period}
**Focus:** {geography_focus}

## 📚 Contenu

### Cartes Historiques
Cartes montrant évolutions territoriales et événements géographiques.

### Reconstructions
Visualisations de sites historiques dans leur état d'origine.

### Géographie Thématique
Cartes thématiques (climat, relief, régions).

### Timelines
Frises chronologiques illustrées.

## 💡 Utilisation
Ces ressources sont conçues pour l'enseignement collège/lycée.
Adaptez selon le niveau de vos élèves.
"""
    
    with open(pack_dir / 'README.md', 'w', encoding='utf-8') as f:
        f.write(readme)
    
    with open(pack_dir / 'pack_index.json', 'w', encoding='utf-8') as f:
        json.dump(pack_index, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ Pack exporté: {pack_dir.name}")
    print(f"📁 {len(pack_index['resources'])} ressources incluses")
else:
    print("\n⏭️  Export désactivé")

print("\n" + "=" * 50)
print("✅ Notebook Histoire-Géographie terminé!")
print("🗺️  Ressources pédagogiques prêtes à l'emploi")
print("=" * 50)